The data contains primarily prices, room descriptions and customer reviews of hotels in  Berlin, London, Madrid, Paris and Rome. 
The goal of this project is to extract and transform data from json files in a way which will enable easily performing comparisons between prices of hotels of different standards (ratings) or between different cities. 

- The user of the data should be able to load the data with the simple call to read_csv or by double clicking and using spreadsheet.

In [42]:
import json
from typing import List
import sys
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sys.path.append('/home/krzysiek/Desktop/projects_etc/football/football_new')
sys.path.append(r'C:\Users\Predator\Desktop\soccer ML\football_new')
sys.path.append(r'C:\Users\Predator')
from slicing_diction import slicing_diction
from intersection import intersection_of_n_sets
from eda_automations import *
from datetime import datetime
from pandas.io.json import json_normalize
import numpy as np
import glob
import re
import os
import requests

<center>Getting the data from github</center>

In [13]:


url = 'https://github.com/KCristopher/data_cleaning/raw/main/hotels/Berlin.json'

response = requests.get(url)

if response.status_code == 200:
    json_data = json.loads(response.text)
    berlin_hotels = pd.json_normalize(json_data, sep='_')
    # Perform operations with the normalized data
    print(berlin_hotels.head())
else:
    print("Error accessing the URL:", response.status_code)

                                        airbnbHotels  \
0  [{'thumbnail': 'https://a0.muscache.com/im/pic...   

                                       bookingHotels  \
0  [{'thumbnail': 'https://cf.bstatic.com/xdata/i...   

                                     hotelsComHotels  
0  [{'title': 'Moxy Berlin Ostbahnhof', 'isAd': T...  


<center>Examining the raw data</center>

In [14]:
berlin_hotels.head()

,airbnbHotels,bookingHotels,hotelsComHotels
0,[{'thumbnail': 'https://a0.muscache.com/im/pic...,[{'thumbnail': 'https://cf.bstatic.com/xdata/i...,"[{'title': 'Moxy Berlin Ostbahnhof', 'isAd': T..."


We can see that the values of the df are the lists of dictionaries.

In [15]:
berlin_hotels.iloc[0, 0][0 : 2]

[{'thumbnail': 'https://a0.muscache.com/im/pictures/miso/Hosting-647664199858827562/original/cfc2fc4c-d703-4827-bc25-f1acb07e0025.jpeg?im_w=720',
  'title': 'Private room in Tempelhof',
  'subtitles': ['Privatzimmer in Tempelhofer Feld', '1 bed', 'Jul 24 – 31'],
  'price': {'currency': '$', 'value': 31, 'period': 'night'},
  'rating': 5,
  'link': 'https://www.airbnb.com/rooms/647664199858827562'},
 {'thumbnail': 'https://a0.muscache.com/im/pictures/b9cb8b8c-51b3-46c4-b9cd-d27053f7d628.jpg?im_w=720',
  'title': 'Private room in Mitte',
  'subtitles': ['Tiny, individual Room with private Bathroom',
   '1 small double bed',
   'Sep 1 – 8'],
  'price': {'currency': '$', 'value': 40, 'period': 'night'},
  'rating': 4.96,
  'link': 'https://www.airbnb.com/rooms/41220512'}]

The dictionaries in each value of the df contains attributes of the hotel rooms, such as price or rating, as keys and the corresponding values as values.
The value can be int, float, str, list or dict.

In [16]:
berlin_hotels.shape

(1, 3)

In [17]:
def extract_values_of_single_row_df(df, col_idx = None) -> list :
    
    """
    Takes a single row df whose values are the lists of dictionaries
    and extracts those lists of dictionaries either from single column
    or from all columns.
    
    Parameters
    ----------
    df : pd.DataFrame.
    col_idx : None or int. Defaults to None.
              The index of the column to extract
              data from.
    Examples
    --------
    >>> extract_values_of_single_row_df(df = berlin_hotels, col_idx = 0)

        [ {'thumbnail': 'https://a0.muscache.com/im/pictures/miso/Hosting-647664199858827562/original/cfc2fc4c-d703-4827-bc25-f1acb07e0025.jpeg?im_w=720',
            'title': 'Private room in Tempelhof',
            'subtitles': ['Privatzimmer in Tempelhofer Feld', '1 bed', 'Jul 24 – 31'],
            'price': {'currency': '$', 'value': 31, 'period': 'night'},
            'rating': 5,
            'link': 'https://www.airbnb.com/rooms/647664199858827562'},
            {'thumbnail': 'https://a0.muscache.com/im/pictures/b9cb8b8c-51b3-46c4-b9cd-d27053f7d628.jpg?im_w=720',
            'title': 'Private room in Mitte',
            'subtitles': ['Tiny, individual Room with private Bathroom',
            '1 small double bed',
            'Sep 1 – 8'],
            'price': {'currency': '$', 'value': 40, 'period': 'night'},
            'rating': 4.96,
            'link': 'https://www.airbnb.com/rooms/41220512'}, ... ]
    Returns
    -------
    list. A list of dictionaries which are inside the value of the first row column col_idx of the df
          if col_idx is not None. 
          Otherwise, a list of all dictionaries inside all values of the first row of the df.
    """
    
    if col_idx is None :
        nest_iterab = [df.iloc[0, i] for i in range(df.shape[1])]
        flat_iterab = [elem for iterab in nest_iterab for elem in iterab]
        
        return flat_iterab
    
    return df.iloc[0, col_idx]

In [18]:
def get_keys_of_jsons(jasons_iterab) :
    
    """
    Takes a list of json files (dictionaries) and
    returns a list of outermost keys of the 
    jsons in that list.
    

    Parameters
    ----------
    jasons_iterab : list. A list of json files.
    
    Returns
    list. List of outermost keys of the jsons in the list.
    
    Example
    -------
    json_iterab = berlin_hotels.iloc[0, 2]
    json_iterab[ : 1]
        
        [{'title': 'Moxy Berlin Ostbahnhof',
          'isAd': True,
          'location': 'Friedrichshain',
          'snippet': {'title': 'Lifestyle Hotel close to Ostbahnhof',
           'text':'Stylish, pet-friendly hotel in Friedrichshain-Kreuzberg connected to...'},
          'paymentOptions': [],
          'highlightedAmenities': [],
          'price': {'currency': '$', 'value': 107, 'withTaxesAndCharges': 120},
          'rating': {'score': 8.8, 'reviews': 596},
          'link': 'https://www.hotels.com/ho497828896/moxy-berlin-ostbahnhof-berlin-germany/'}]
        
        
    get_keys_of_jsons(json_iterab)
    
        
        ['title',
         'isAd',
         'location',
         'snippet',
         'paymentOptions',
         'highlightedAmenities',
         'price',
         'rating',
         'link']
    """
    
    keys = []

    for b in jasons_iterab :
        for k in b.keys() :
            if k not in keys :
                keys.append(k)
    return keys

In [19]:
def get_keys_of_jsons_in_single_row_df(df, col_idx) -> list :
    
    """
    Parameters
    ----------
    df : pd.DataFrame.
    col_idx : int. The index of the column to extract.
    
    Returns
    -------
    list. A list of outermost keys of the jsons in the list.

    Example
    -------
    get_keys_of_jsons_in_single_row_df(df = berlin_hotels, col_idx = 2)

        ['title',
        'isAd',
        'location',
        'snippet',
        'paymentOptions',
        'highlightedAmenities',
        'price',
        'rating',
        'link']
    """
    
    jsons_iterab = extract_values_of_single_row_df(df = df, col_idx = col_idx)
    keys = get_keys_of_jsons(jsons_iterab)
    
    return keys
    
    

The problem is that in every column there are data from different hotels, and the jasons have different keys.
So if we try to make one big df out of it, some jsons will have 0 length values, and therefore we get ValueError when creating a df. So we need to create separate df for every column.

In [20]:
def get_df_from_jsons_in_single_col_in_single_row_df(df, col_idx,\
                                                     include_nans = True) -> pd.DataFrame :
    
    """
    Parameters
    ----------
    df : pd.DataFrame.
    col_idx : None or int. Defaults to None.
              The index of the column to extract
              data from.
    include_nans : bool. Defaults to True.
    
    Returns
    -------
    pd.DataFrame.
    """
    
    keys = get_keys_of_jsons_in_single_row_df(df = df, col_idx = col_idx)
    blank_vals = [ [ ] for _ in range(len(keys))]
    diction = dict(zip(keys, blank_vals))

    for b in df.iloc[0, col_idx] :
        for key, val in b.items() :
            diction[key].append(val)
    
    lengths_by_keys = {k : len(v) for k, v in diction.items()}
    lengths_max = max([v for v in lengths_by_keys.values()])
    
    if include_nans :
        pairs_with_shorter_vals = {k : v for k, v in lengths_by_keys.items()\
                                  if v < lengths_max}
        
        pairs_shorter_vals_deficit = {k : lengths_max - v for k, v in\
                                      pairs_with_shorter_vals.items()}
        
        for k, v in pairs_shorter_vals_deficit.items() :
            diction[k] = diction[k] + [np.nan for _ in range(v)]
    else :
        for k in keys_with_shorter_vals :
            del diction[k]
    
    return pd.DataFrame(diction)

In [21]:
def expand_dictionary_column(df_col) :
    
    """
    Parameters
    ----------
    df_col : pd.Series.
    
    Returns
    -------
    pd.DataFrame.
    
    Example
    -------
    print(df)
    
            column_with_dict
        0  {'currency': '$', 'value': 31, 'period': 'night'}
        1    {'currency': '€', 'value': 45, 'period': 'day'}
        2  {'currency': '$', 'value': 17, 'period': 'night'
    
    print(expand_dictionary_column(df['column_with_dict']))
       

          column_with_dict_currency  column_with_dict_value column_with_dict_period  
        0                         $                      31                   night  
        1                         €                      45                     day  
        2                         $                      17                   night
    
    """
    # Extract the column values as a Series
    column = df_col

    # Get all unique keys from the dictionaries
    all_keys = set().union(*(d.keys() for d in column))
    
    df = pd.DataFrame()

    # Create new columns for each key in the dictionary
    for key in all_keys:
        new_column_name = f"{df_col.name}_{key}"
        df[new_column_name] = [d.get(key) for d in column]

    return df

def expand_diction_columns_inside_df(df, col_names : list) :
    
    """
    Parameters
    ----------
    df : pd.DataFrame.
    col_names : list.

    Returns
    -------
    pd.DataFrame.
    
    """
    
    new_frames = [expand_dictionary_column(df[cn]) for cn in col_names]
    new_big_frame = pd.concat([df, pd.concat(new_frames, axis = 1)], axis = 1)
    
    
    return new_big_frame

def find_dictionary_columns(df) -> list :
    
    """
    Parameters
    ----------
    df : pd.DataFrame.
    
    Returns
    -------
    list.
    
    Example
    -------
    find_dictionary_columns(df)
        
        ['price']

    """
    
    dictionary_columns = []

    # Iterate over each column
    for column in df.columns :
        if df[column].dtype == 'object' :
            is_dictionary_column = all(value.startswith('{')\
                                       and value.endswith('}')\
                                       for value in df[column].astype(str))
            if is_dictionary_column :
                dictionary_columns.append(column)

    return dictionary_columns

def get_clean_df_from_jsons_in_single_col_in_single_row_df(df, col_idx, include_nans = True, flatten_iterab_cols = False,\
                                                           remove_diction_cols = False) :
    
    """
    Parameters
    ----------
    df : pd.DataFrame.
    col_idx : None or int. Defaults to None.
              The index of the column to extract
              data from.
    include_nans : bool. Defaults to True.
    flatten_iterab_cols : bool. Defaults to False.
    remove_diction_cols : bool. Defaults to False.
    
    Returns
    -------
    pd.DataFrame.
    """
    
    frame = get_df_from_jsons_in_single_col_in_single_row_df(df = df, col_idx = col_idx,\
                                                             include_nans = include_nans)
    
    diction_cols = find_dictionary_columns(df = frame)
    new_frame = expand_diction_columns_inside_df(df = frame, col_names = diction_cols)

    if flatten_iterab_cols :
        object_cols = [c for c in new_frame.columns if new_frame[c].dtype == 'object']
        iterab_cols = [c for c in object_cols if any([isinstance(v, list) for v in new_frame[c]])]
        for ic in iterab_cols :
            new_frame[ic] = new_frame[ic].apply(lambda iterab : iterab[0])
    
    if remove_diction_cols :
        new_frame = new_frame.drop(diction_cols, axis = 1)

    return new_frame

def get_clean_frames_from_jsons_in_single_row_df(df, include_nans = True, remove_diction_cols = False) :
    
    """
    Parameters
    ----------
    df : pd.DataFrame.
    col_idx : None or int. Defaults to None.
              The index of the column to extract
              data from.
    include_nans : bool. Defaults to True.
    
    Returns
    -------
    tuple. Tuple of dataframes, one for each column in input df.
    """
    
    return tuple([get_clean_df_from_jsons_in_single_col_in_single_row_df(df = df, col_idx = i, remove_diction_cols = remove_diction_cols)\
                 for i in range(df.shape[1])])
    

In [22]:
def deiterab_feature_in_df(iterab_feature : str, df) :
        
        """
        Parameters
        ----------
        iterab_feature : str. The name of the column
                        in the df which contains the
                        iterables (lists).
        df : pd.DataFrame.
        
        Returns
        -------
        pd.DataFrame.
        """
        
        df = df.copy()
        df[iterab_feature] = df[iterab_feature].apply(lambda iterab : iterab[0] if len(iterab) > 0 else np.nan)
        
        return df

def deiterab_features_in_df(iterab_features : list, df) :
    
    """
    Parameters
    ----------
    iterab_features : list. The names of the columns
                      in the df which contain the
                      iterables (lists).
    df : pd.DataFrame.
    
    Returns
    -------
    pd.DataFrame.

    Example
    -------
    deiterab_features_in_df( ['subtitles'] , airbnb).head()
    """
    
    df = df.copy()
    
    for ife in iterab_features :
        deiterabbed_feature = deiterab_feature_in_df(iterab_feature = ife, df = df)[ife].values
        df[ife] = deiterabbed_feature
 
    return df

In [23]:
def replace_string_with_nans(df, string) :
    
    """
    Parameters
    ----------
    df : pd.DataFrame.
    string : str.
    
    Returns
    -------
    pd.DataFrame.

    Example
    -------
    replace_string_with_nans(airbnb, 'No rating')['rating'].head()

        0    5.00
        1    4.96
        2    4.69
        3    4.89
        4    4.87
        Name: rating, dtype: float64

    """
    
    df = df.copy()
    df = df.replace(string, np.nan)
    
    return df

def replace_strings_with_nans(df, strings_iterab) :
    
    """
    Parameters
    ----------
    df : pd.DataFrame.
    strings_iterab : list.
    
    Returns
    -------
    pd.DataFrame.
    
    Example
    -------
    """

    df = df.copy()

    for si in strings_iterab :
        df = replace_string_with_nans(df = df, string = si)
    
    return df

Checking which attributes (like rating or price) of the hotel rooms appear on all travel agencies websites (booking, airbnb, hotels.com), so that we can create a dataset based on shared attributes from all three websites to enable making comparisons of offers from different travel agencies.

In [24]:
def intersection_of_dfs(dataframes_iterab : List[pd.DataFrame]) -> pd.DataFrame :

    """
    Parameter
    ----------
    dataframes_iterab : list of DataFrames.

    Returns
    -------
    pd.DataFrame.

    Example
    -------
    print(booking.columns)
        
        Index(['thumbnail', 'title', 'stars', 'preferredBadge', 'promotedBadge',
            'location', 'subwayAccess', 'sustainability', 'distanceFromCenter',
            'highlights', 'link', 'price_value', 'price_taxesAndCharges',
            'price_currency', 'rating_score', 'rating_scoreDescription',
            'rating_reviews', 'marketplace'],
        dtype='object')
    
    print(airbnb.columns)

        Index(['thumbnail', 'title', 'subtitles', 'rating', 'link', 'price_value',
            'price_period', 'price_currency', 'marketplace'],
        dtype='object')
    
    print(hotelscom.columns)

        Index(['title', 'isAd', 'location', 'paymentOptions', 'highlightedAmenities',
            'link', 'snippet_title', 'snippet_text', 'price_value',
            'price_withTaxesAndCharges', 'price_currency', 'rating_score',
            'rating_reviews', 'marketplace'],
        dtype='object')

    # the output will have only the columns that are present in all three dataframes
    
    print(intersection_of_dfs([airbnb, booking, hotelscom]).head())
        
        link                                           marketplace price_currency  \
    0  https://www.airbnb.com/rooms/647664199858827562      airbnb              $   
    1            https://www.airbnb.com/rooms/41220512      airbnb              $   
    2            https://www.airbnb.com/rooms/30881310      airbnb              $   
    3             https://www.airbnb.com/rooms/2591901      airbnb              $   
    4            https://www.airbnb.com/rooms/19075304      airbnb              $   

    price_value                                       title  
    0           31                   Private room in Tempelhof  
    1           40                       Private room in Mitte  
    2          108               Hotel room in Prenzlauer Berg  
    3           34                 Private room in Lichtenberg  
    4           71  Private room in Charlottenburg-Wilmersdorf  
      
        """

    common_cols = intersection_of_n_sets([{c for c in df.columns} for df in dataframes_iterab])
    common_cols = sorted(list(common_cols))
    
    return pd.concat([df[common_cols] for df in dataframes_iterab], axis = 0)

In [25]:
airbnb, booking, hotelscom = get_clean_frames_from_jsons_in_single_row_df(berlin_hotels, remove_diction_cols = True)

In [26]:
airbnb['marketplace'] = 'airbnb'
booking['marketplace'] = 'booking'
hotelscom['marketplace'] = 'hotelscom'

In [27]:
airbnb.head()

,thumbnail,title,subtitles,rating,link,price_period,price_currency,price_value,marketplace
0,https://a0.muscache.com/im/pictures/miso/Hosti...,Private room in Tempelhof,"[Privatzimmer in Tempelhofer Feld, 1 bed, Jul ...",5,https://www.airbnb.com/rooms/647664199858827562,night,$,31,airbnb
1,https://a0.muscache.com/im/pictures/b9cb8b8c-5...,Private room in Mitte,"[Tiny, individual Room with private Bathroom, ...",4.96,https://www.airbnb.com/rooms/41220512,night,$,40,airbnb
2,https://a0.muscache.com/im/pictures/miso/Hosti...,Hotel room in Prenzlauer Berg,[Serviced Apartment - Studio in Prenzlauer Ber...,4.69,https://www.airbnb.com/rooms/30881310,night,$,108,airbnb
3,https://a0.muscache.com/im/pictures/63522495/a...,Private room in Lichtenberg,"[City East perfectly located, 1 king bed, Jul ...",4.89,https://www.airbnb.com/rooms/2591901,night,$,34,airbnb
4,https://a0.muscache.com/im/pictures/miso/Hosti...,Private room in Charlottenburg-Wilmersdorf,[Musical place next to Bundesplatz (U9 & S-Rin...,4.87,https://www.airbnb.com/rooms/19075304,night,$,71,airbnb


In [28]:
airbnb, booking, hotelscom =  deiterab_features_in_df(['subtitles'], airbnb), deiterab_features_in_df(['highlights'], booking),\
    deiterab_features_in_df(['paymentOptions', 'highlightedAmenities'], hotelscom)

In [46]:
airbnb, booking, hotelscom = replace_strings_with_nans(airbnb, ['No rating']), replace_strings_with_nans(booking, ['No rating']),\
    replace_strings_with_nans(hotelscom, ['No rating'])

In [76]:
def check_if_any_value_of_feature_is_none(df, feature) :
    
    """
    Parameters
    ----------
    df : pd.DataFrame.
    feature : str.
    
    Returns
    -------
    bool.
    
    Example
    -------
    check_if_any_value_of_feature_is_none(airbnb, 'rating')
    
        False
    """
    
    return any([v is None for v in df[feature].values])



def check_whether_values_of_features_are_none(df : pd.DataFrame) -> dict :

    return {c : check_if_any_value_of_feature_is_none(df, c) for c in df.columns}

def replace_none_with_nans_in_df(df : pd.DataFrame) -> pd.DataFrame :

    """
    Parameters
    ----------
    df : pd.DataFrame.

    Returns
    -------
    pd.DataFrame.

    Example
    -------
    """

    relev_features = [c for c, v in check_whether_values_of_features_are_none(df).items() if v == True]
    data = df[relev_features].copy()

    for c in data.columns :
        data[c] = data[c].apply(lambda v : np.nan if v is None else v)
    
    other_cols = [c for c in df.columns if c not in relev_features]
    data = pd.concat([data, df[other_cols].copy()], axis = 1)
    data = data[[c for c in df.columns]]

    
    return data

,title,isAd,location,paymentOptions,highlightedAmenities,link,snippet_text,snippet_title,price_withTaxesAndCharges,price_currency,price_value,rating_score,rating_reviews,marketplace
0,Moxy Berlin Ostbahnhof,True,Friedrichshain,NaN,NaN,https://www.hotels.com/ho497828896/moxy-berlin...,"Stylish, pet-friendly hotel in Friedrichshain-...",Lifestyle Hotel close to Ostbahnhof,120,$,107,8.8,596.0,hotelscom
1,Sheraton Berlin Grand Hotel Esplanade,True,Tiergarten,NaN,NaN,https://www.hotels.com/ho109823/sheraton-berli...,An urban oasis of sleek design and intuitive s...,Berlin hotel with panoramic windows,183,$,163,8.6,1000.0,hotelscom
2,IntercityHotel Berlin Ostbahnhof,False,Friedrichshain,NaN,NaN,https://www.hotels.com/ho178251/intercityhotel...,NaN,NaN,105,$,93,8.4,1005.0,hotelscom
3,URBAN LOFT Berlin,False,Berlin,NaN,NaN,https://www.hotels.com/ho2505665600/urban-loft...,NaN,NaN,97,$,86,9.0,154.0,hotelscom
4,Dorint Kurfürstendamm Berlin,False,Berlin,Fully refundable,NaN,https://www.hotels.com/ho185137/dorint-kurfurs...,NaN,NaN,166,$,148,9.2,1006.0,hotelscom
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,Hotel Brandenburger Tor Potsdam,False,"Potsdam, 14.96 mi from Berlin",NaN,NaN,https://www.hotels.com/ho375925/hotel-brandenb...,NaN,NaN,137,$,123,9.0,161.0,hotelscom
496,Hotel am Luisenplatz,False,"Potsdam, 15.03 mi from Berlin",NaN,NaN,https://www.hotels.com/ho212132/hotel-am-luise...,NaN,NaN,168,$,150,8.4,213.0,hotelscom
497,MAXX Hotel Sanssouci Potsdam,False,"Potsdam, 15.03 mi from Berlin",NaN,NaN,https://www.hotels.com/ho184477/maxx-by-steige...,NaN,NaN,96,$,86,8.2,916.0,hotelscom
498,Bed & Breakfast am Luisenplatz,False,"Potsdam, 15.05 mi from Berlin",Fully refundable,NaN,https://www.hotels.com/ho1479501728/bed-breakf...,NaN,NaN,164,$,146,9.2,29.0,hotelscom


In [77]:
airbnb, booking, hotelscom = [*map(replace_none_with_nans_in_df, [airbnb, booking, hotelscom])]

In [86]:
hotelscom.select_dtypes('object').head()

,title,location,paymentOptions,highlightedAmenities,link,snippet_text,snippet_title,price_currency,marketplace
0,Moxy Berlin Ostbahnhof,Friedrichshain,NaN,NaN,https://www.hotels.com/ho497828896/moxy-berlin...,"Stylish, pet-friendly hotel in Friedrichshain-...",Lifestyle Hotel close to Ostbahnhof,$,hotelscom
1,Sheraton Berlin Grand Hotel Esplanade,Tiergarten,NaN,NaN,https://www.hotels.com/ho109823/sheraton-berli...,An urban oasis of sleek design and intuitive s...,Berlin hotel with panoramic windows,$,hotelscom
2,IntercityHotel Berlin Ostbahnhof,Friedrichshain,NaN,NaN,https://www.hotels.com/ho178251/intercityhotel...,NaN,NaN,$,hotelscom
3,URBAN LOFT Berlin,Berlin,NaN,NaN,https://www.hotels.com/ho2505665600/urban-loft...,NaN,NaN,$,hotelscom
4,Dorint Kurfürstendamm Berlin,Berlin,Fully refundable,NaN,https://www.hotels.com/ho185137/dorint-kurfurs...,NaN,NaN,$,hotelscom


In [87]:
hotelscom.sample(100).head(50)

,title,isAd,location,paymentOptions,highlightedAmenities,link,snippet_text,snippet_title,price_withTaxesAndCharges,price_currency,price_value,rating_score,rating_reviews,marketplace
389,Limehome Berlin Leberstraße 6,False,Entire apartment,Fully refundable,NaN,https://www.hotels.com/ho623605088/limehome-be...,NaN,NaN,184,$,159,8.0,9.0,hotelscom
87,aletto Hotel Potsdamer Platz,False,Kreuzberg,NaN,NaN,https://www.hotels.com/ho1453979616/aletto-hot...,NaN,NaN,107,$,96,9.2,315.0,hotelscom
73,Leonardo Hotel Berlin,False,Berlin,NaN,NaN,https://www.hotels.com/ho311565/leonardo-hotel...,NaN,NaN,97,$,87,8.6,1001.0,hotelscom
88,Mercure Hotel Berlin Mitte,False,Kreuzberg,NaN,NaN,https://www.hotels.com/ho203187/mercure-hotel-...,NaN,NaN,149,$,133,8.6,977.0,hotelscom
105,Yggotel Spurv,False,Wilmersdorf,NaN,NaN,https://www.hotels.com/ho218533/novum-hotel-li...,NaN,NaN,95,$,85,9.0,16.0,hotelscom
64,Hotel AMANO Grand Central,False,Berlin,NaN,NaN,https://www.hotels.com/ho527834/hotel-amano-gr...,NaN,NaN,108,$,96,8.8,1004.0,hotelscom
466,ipartment Berlin Airport,False,"Schoenefeld, 12.47 mi from Berlin",NaN,Kitchen,https://www.hotels.com/ho2679500256/ipartment-...,NaN,NaN,99,$,92,8.0,1.0,hotelscom
368,Hotel-Pension Funk,False,Berlin,Fully refundable,NaN,https://www.hotels.com/ho458549/hotel-pension-...,NaN,NaN,137,$,122,8.2,56.0,hotelscom
116,the niu Hide,False,Lichtenberg,"Reserve now, pay later",NaN,https://www.hotels.com/ho804756640/the-niu-hid...,NaN,NaN,90,$,80,9.0,438.0,hotelscom
397,Limehome Berlin Kopenhagener Straße,False,Entire apartment,Fully refundable,NaN,https://www.hotels.com/ho579031/limehome-berli...,NaN,NaN,297,$,214,8.8,39.0,hotelscom


In [88]:
for df in [airbnb, booking, hotelscom] :
    df['price_currency'] = df['price_currency'].apply(lambda s : s.replace('US', ''))

In [89]:
comparison_frame = intersection_of_dfs([airbnb, booking, hotelscom])

In [91]:
comparison_frame.sample(n = 50).head(10)

,link,marketplace,price_currency,price_value,title
214,https://www.hotels.com/ho200289/alfa-hotel-ber...,hotelscom,$,86,Alfa Hotel
354,https://www.hotels.com/ho130537/the-mandala-su...,hotelscom,$,196,The Mandala Suites Berlin
349,https://www.hotels.com/ho1116569664/gastehaus-...,hotelscom,$,130,Gästehaus Lazarus
309,https://www.booking.com/hotel/de/axel-berlin.h...,booking,$,103,Axel Hotel Berlin-Adults Only
286,https://www.hotels.com/ho589073/hotel-garni-an...,hotelscom,$,81,Hotel Garni An der Weide
311,https://www.hotels.com/ho442447/hotel-nikolai-...,hotelscom,$,162,Hotel Nikolai Residence
346,https://www.booking.com/hotel/de/grand-hyatt-b...,booking,$,251,Grand Hyatt Berlin
496,https://www.booking.com/hotel/de/top-spot-mitt...,booking,$,123,TOP SPOT MITTE . THE PLACE TO BE !
134,https://www.booking.com/hotel/de/hotpenskastan...,booking,$,133,Hotel Kastanienhof
176,https://www.airbnb.com/rooms/760108563556107857,airbnb,$,63,Private room in Berlin


In [92]:
comparison_frame.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1270 entries, 0 to 499
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   link            1270 non-null   object
 1   marketplace     1270 non-null   object
 2   price_currency  1270 non-null   object
 3   price_value     1270 non-null   int64 
 4   title           1270 non-null   object
dtypes: int64(1), object(4)
memory usage: 59.5+ KB


In [93]:
curr_time = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')

for df in [airbnb, booking, hotelscom] :
    marketplace = df.marketplace.iloc[0]
    file_name = '{}_{}'.format(marketplace, curr_time)
    path = 'Berlin/{}.csv'.format(file_name)
    df.to_csv(path, index = False)

In [94]:
curr_time = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')

comparison_frame.to_csv('Berlin/comparison_frame_{}.csv'.format(curr_time), index = False)